In [10]:
import pandas as pd 
import sys
import os
sys.path.append(os.path.abspath(r'C:\Users\Yibabe\Desktop\kifiyaAIMweek-2\scripts'))
from task1_connection import load_data_from_postgres


In [11]:
# Define the query to fetch data from the 'cleaned_data' table
query = "SELECT * FROM cleaned_data"

# Load the data into a DataFrame
df_cleaned = load_data_from_postgres(query)

# Display the first few rows of the DataFrame to verify
df_cleaned.head()


C:\Users\Yibabe\Desktop\kifiyaAIMweek-2\scripts\task1_connection.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,Dur. (ms),Start,End,IMSI,MSISDN/Number,Email DL (Bytes),Gaming DL (Bytes),Google DL (Bytes),Avg RTT DL (ms),Avg Bearer TP DL (kbps),...,HTTP DL (Bytes),HTTP UL (Bytes),Total DL (Bytes),Total UL (Bytes),Social Media DL (Bytes),Youtube DL (Bytes),Netflix DL (Bytes),Handset Type,Handset Manufacturer,Bearer Id
0,951292.0,4/14/2019 2:11,4/25/2019 2:26,2.082010e+14,33763490140.0,2987408.0,748906189.0,1304524.0,39.0,3698.0,...,230583.0,25427.0,775350343.0,58813016.0,3103940.0,12189786.0,6858496.0,undefined,undefined,1.304243e+19
1,941634.0,4/14/2019 12:48,4/25/2019 10:22,2.082014e+14,33698743617.0,3014208.0,801543830.5,7560820.0,45.0,46.0,...,0.0,0.0,861612261.0,42363146.0,342043.0,18436641.0,10587264.0,Samsung Galaxy A8 (2018),Samsung,1.311448e+19
2,864482.0,4/15/2019 0:32,4/25/2019 0:40,2.082003e+14,33659219748.0,522457.0,801543830.5,8601949.0,97.0,3845.0,...,132246.0,32449.0,850570347.0,22417975.0,1541259.0,16338345.0,12211149.0,Samsung Galaxy Note 8 (Sm-N950F Ds),Samsung,1.304243e+19
3,848637.0,4/15/2019 10:30,4/25/2019 6:14,2.082098e+14,33665646348.0,3095404.0,565283082.0,4366381.0,45.0,6.0,...,0.0,0.0,594967693.0,56559490.0,2653333.0,2595678.0,16973815.0,undefined,undefined,7.277826e+18
4,889834.0,4/15/2019 11:33,4/25/2019 18:44,2.082019e+14,33664473872.0,1541381.0,504604946.0,7241855.0,217.0,28305.0,...,35986300.0,689004.0,535118044.0,33691738.0,829305.0,2805710.0,18094847.0,Huawei B593S-22,Huawei,1.311448e+19


In [12]:
df_cleaned.shape

(148992, 22)

In [13]:
# Count number of xDR sessions per user
session_counts = df_cleaned.groupby('IMSI')['Bearer Id'].count().reset_index(name='Number of xDR Sessions')
print("Number of xDR Sessions per User:")
print(session_counts.head())


Number of xDR Sessions per User:
           IMSI  Number of xDR Sessions
0  2.040471e+14                       1
1  2.040808e+14                       1
2  2.082001e+14                       1
3  2.082001e+14                       1
4  2.082001e+14                       2


In [14]:
# Calculate total session duration per user
total_durations = df_cleaned.groupby('IMSI')['Dur. (ms)'].sum().reset_index(name='Total Session Duration (ms)')
print("Total Session Duration per User:")
print(total_durations.head())


Total Session Duration per User:
           IMSI  Total Session Duration (ms)
0  2.040471e+14                     869844.0
1  2.040808e+14                     877385.0
2  2.082001e+14                      86019.0
3  2.082001e+14                      86254.0
4  2.082001e+14                     573923.0


In [15]:
# Calculate total download and upload data per user
total_dl_ul = df_cleaned.groupby('IMSI').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
}).reset_index()
total_dl_ul.rename(columns={'Total DL (Bytes)': 'Total Download (Bytes)', 'Total UL (Bytes)': 'Total Upload (Bytes)'}, inplace=True)
print("Total Download and Upload Data per User:")
print(total_dl_ul.head())


Total Download and Upload Data per User:
           IMSI  Total Download (Bytes)  Total Upload (Bytes)
0  2.040471e+14              78697597.0            60456049.0
1  2.040808e+14             194828056.0            37295915.0
2  2.082001e+14              31376981.0            53794658.0
3  2.082001e+14             392918313.0            33583272.0
4  2.082001e+14             961767023.0            71325004.0


In [16]:
# Calculate total data volume per application per user
applications = ['Social Media DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
app_data_volumes = df_cleaned.groupby('IMSI').agg({
    **{app: 'sum' for app in applications}
}).reset_index()

print("Total Data Volume per Application per User:")
print(app_data_volumes.head())


Total Data Volume per Application per User:
           IMSI  Social Media DL (Bytes)  Youtube DL (Bytes)  \
0  2.040471e+14                1500145.0          18980320.0   
1  2.040808e+14                 668596.0           9839889.0   
2  2.082001e+14                3073697.0           5556563.0   
3  2.082001e+14                2542767.0          22076208.8   
4  2.082001e+14                3326811.0           4046597.0   

   Netflix DL (Bytes)  Google DL (Bytes)  Email DL (Bytes)  Gaming DL (Bytes)  \
0          21621336.0          5985089.0         2518425.0         42060209.5   
1          10340908.0          8572779.0          842279.0        164563605.0   
2          15305361.0           603691.0         3076735.0         42060209.5   
3          10065639.0         10699827.0         3248863.0        343946503.0   
4          30725208.9          9118088.0         4225757.0        910301321.0   

   Other DL (Bytes)  
0      3.891597e+08  
1      4.676781e+08  
2      5.000445e+0

In [17]:
# Merge all aggregated data into one DataFrame
user_overview = session_counts.merge(total_durations, on='IMSI')
user_overview = user_overview.merge(total_dl_ul, on='IMSI')
user_overview = user_overview.merge(app_data_volumes, on='IMSI')

print("User Overview DataFrame:")
print(user_overview.head())


User Overview DataFrame:
           IMSI  Number of xDR Sessions  Total Session Duration (ms)  \
0  2.040471e+14                       1                     869844.0   
1  2.040808e+14                       1                     877385.0   
2  2.082001e+14                       1                      86019.0   
3  2.082001e+14                       1                      86254.0   
4  2.082001e+14                       2                     573923.0   

   Total Download (Bytes)  Total Upload (Bytes)  Social Media DL (Bytes)  \
0              78697597.0            60456049.0                1500145.0   
1             194828056.0            37295915.0                 668596.0   
2              31376981.0            53794658.0                3073697.0   
3             392918313.0            33583272.0                2542767.0   
4             961767023.0            71325004.0                3326811.0   

   Youtube DL (Bytes)  Netflix DL (Bytes)  Google DL (Bytes)  \
0          18980320.0